#Open_domain_search_assistant

50483 - Joao Diogo


53044 - Gonçalo Antunes


53600 - Samuel Viegas

###Preparar workspace, modelos e dependências

In [ ]:
# Colab Setup
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# After downloading the shared starting point folder as a Zip
# Unzip it and re-upload it to a location on your GDrive

# This command copies the contents from the folder you uploaded to GDrive, to the colab working dir
!cp -r /content/drive/MyDrive/PW /content

# Add working dir to the sys path, so that we can find the aux python files when running the Notebook
import sys
if not '/content/ProjectoPW2021' in sys.path:
  sys.path += ['/content/ProjectoPW2021']

import numpy as np
import pandas as pd
import pickle
import urllib
from scipy.sparse import csr_matrix
from time import time
!pip install scikit-network
#from sknetwork.visualization import svg_graph, svg_digraph, svg_bigraph
from sklearn.decomposition import randomized_svd
from IPython.display import SVG
from sknetwork.visualization import svg_graph
import pprint

#import wikipedia_principal_eigenvector as pagerank

#import the BLINK/DBpedia-spotlight
#because DBpedia-spotlight is REST based we need to import urls and network librarys to make requests
import requests
!pip install transformers
from transformers import BartModel, BartConfig, pipeline
# Initializing a BART facebook/bart-large style configuration
configuration = BartConfig()
# Initializing a model from the facebook/bart-large style configuration
model = BartModel(configuration)
# Accessing the model configuration
configuration = model.config
summarization = pipeline("summarization")

## 1º passo

1. Preparar o json (`allin.json`) contendo todas as conversas por processar

2. Utilizar o trabalho de RI juntamente com o elasticSearch para ter o Groundtruth File com perguntas e respetivas 10 melhores respostas

</br>

Objeto: `conversa`

</br>

---

## 2º passo

+ Qual o entity linker a usar de modo a extrair entidades? 

  - [DBpedia-spotlight](https://www.dbpedia-spotlight.org/api)

+ Outras hipóteses:

  - [Blinker](https://github.com/facebookresearch/BLINK)


</br>

<ins>Nota:</ins> Inicialmente os resultados não eram `case sensitive`

Exemplo:

| Entidades |1|2|3|4| ... |
|----------:|----------:|----------:|----------:| ----------:|----------:|
| `Tópico 1` |Species| species | tiger shark | bull shark | (...)|

</br>

Objeto: `allMatrixAllDataset`

</br>

---

## 3º passo

Aplicar ao grafo das entidades o algoritmo Pagerank dado nas aulas
[entitys, entitys]-> Pagerank -> [p_results,n]

Pagerank: https://scikit-learn.org/stable/auto_examples/applications/wikipedia_principal_eigenvector.html

TO-DO:

Thresholds a calibrar que não estão na tabela:

1-Na criação do grafo para remover entidades

2-No stopping criteria do PR (não é preciso)

3-Peso das entidades das Queries vs das passagens

Modos de funcionamento:

1-Entities of the passages of the current turn (DONE)

2-Entities of the the passages of the first turn (DONE)

3-Entities of the passages returned in previous turns (DONE)

4-Q entities penalty

## FLAGS

In [ ]:
# false -> Matrix value's are 1s
# true  -> Matrix value's ranged 0.5 - 1.0
MATRIX_FRAC = False

In [ ]:
# Our Python code
## Loading pickles, dbpedia-entity-retrieval, pageRank

!cp '/content/PW/Python/SearchAssistant.py' .
!cp '/content/PW/Python/AssistantTools.py' .

import SearchAssistant
import AssistantTools

# Variables
## Our object, complete topics, complete matrices
assistant = SearchAssistant.getAssistant(MATRIX_FRAC)

conversa = assistant.getConversa()

allMatrixAllDataset = assistant.getAllMatrixAllDataset()

As seguintes funções foram criadas para remover substituir os pronomes.

 Já não é necessário voltar a correr.

In [ ]:
import requests
import json

with open('PW/allIn.json', 'r') as j:
    json_data = json.load(j)

In [ ]:
import csv
tsv_file = open("correto.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")
for row in read_tsv:
  aux = row[0].split("_")
  try:
    json_data[aux[0]][aux[1]]["utterance"] = row[1]
  except:
    continue

In [ ]:
json_data["78"]["6"]["utterance"]

In [ ]:
top10=[]
conversa =[]
pergunta=[]
tudo=[]
for conv in json_data:
  for utNumber in json_data[conv]:
      ut = json_data[conv][utNumber]["utterance"]
      tudo.append(ut)
      for i in json_data[conv][utNumber]["top10"]:
        tudo.append(i)
      pergunta.append(tudo)
      tudo = []
  conversa.append(pergunta)
  pergunta = []    

Definir função para calcular o top10 respostas

In [ ]:
def calcTop10 (transpost_ocurrences,scores,topicNumber,answerNumber):
    top10Scores=[]
    soma=0
    for sentence in transpost_ocurrences.toarray():
      for counter in range(0,len(sentence)):
        if sentence[counter]==1:
          soma += scores[counter]
      #mascara nos elementos que tem 0, só vai contar com os que têm 1
      #print(soma)
      top10Scores.append(soma)
      soma=0
    order=pd.DataFrame(top10Scores[1:11],columns=["Score"])
    #order.insert(0,"Id",np.arange(0,10))
    order = order.sort_values(by='Score', ascending=False)

    data=pd.DataFrame(conversa[topicNumber][answerNumber][1:11],columns=["TOP10RespostasOrdenadas"])
    reordered=data.reindex(order.index)
    reordered.insert(1,"Antigas",conversa[topicNumber][answerNumber][1:11])
    print(reordered)
    return reordered

In [ ]:
def addEntry(anteriores, atual):
  allEntidades = []
  for ant in anteriores:
        antArr = np.array(ant)
        entidades = []
        for principal in antArr:
          if principal[1][0]==1:
              entidades.append(principal[0])
        allEntidades.append(entidades)

  for j in range(len(allEntidades)):
    for k in atual:
      if k[0] in allEntidades[j]:
        k[1]= np.insert(k[1],len(k[1]),1)
      else:
        k[1]= np.insert(k[1],len(k[1]),0)
  return atual

Definir a função que sumariza e retorna a resposta final

In [ ]:
#funcao que sumariza os 3 melhores textos
def sumarize (reordered):
  aux = reordered["TOP10RespostasOrdenadas"].index
  toSumm = reordered["TOP10RespostasOrdenadas"][aux[0]]+reordered["TOP10RespostasOrdenadas"][aux[1]]+reordered["TOP10RespostasOrdenadas"][aux[2]]
  summary_text = summarization(toSumm)[0]['summary_text']
  return summary_text

Definir a função que constrói a matriz e o grafo de entidades 

In [ ]:
#retorna primeiro a matrix de entidades e depois a transposta e depois os scores
def create_EntityMatrix_Transpose_Pagerank(conteudo,threshold=3):
    #extrair os arrays que diz em que documento ocorre cada entidade
    arr = np.array(conteudo)
    aux = []
    for i in arr[:,1]:
      aux.append(i.tolist())
    #criar a matriz de entidades
    entitys = arr[:,0]
    matrix_entitys=csr_matrix(aux)
    array_ocurrences=matrix_entitys
    #transpose the matrix
    transpost_ocurrences=np.transpose(array_ocurrences)
    #Calculate the matrix of entitys
    mult = np.dot(array_ocurrences,transpost_ocurrences)

    #Modificação professor
    graph=mult
    graph_cut = graph > threshold #(graph_cut fica com 0s e 1s)
    clean_graph = np.multiply(graph, graph_cut) #(a multiplicação vai fazer reset aos edges abaixo do threshold)
    clean_graph = np.multiply(graph, (graph > threshold))
    mult=clean_graph
    print(mult.todense())
    #aplicar o pagerank ao grafo
    entity_graph=csr_matrix(mult,dtype=float)
    scores=assistant.computePageRank(entity_graph)

    #Dataframe to debug
    #data=pd.DataFrame(mult.todense(),columns=entitys)
    #data.insert(0,"Entitys",entitys)
    #print(data)

    return entity_graph,transpost_ocurrences,entitys,scores

In [ ]:
#retorna primeiro a matrix de entidades e depois a transposta e depois os scores
def create_EntityMatrix_with1Q_Transpose_Pagerank(conteudo,anteriores,threshold=3):
    #extrair os arrays que diz em que documento ocorre cada entidade
    atual = np.array(conteudo)
    allEntidades =[]

    atual = addEntry(anteriores,atual)

    #Adicinar uma coluna
    aux = []
    for i in atual[:,1]:
      aux.append(i.tolist())
    #criar a matriz de entidades
    entitys = atual[:,0]
    matrix_entitys=csr_matrix(aux)
    array_ocurrences=matrix_entitys
    #transpose the matrix
    transpost_ocurrences=np.transpose(array_ocurrences)
    #Calculate the matrix of entitys
    mult = np.dot(array_ocurrences,transpost_ocurrences)

    #Modificação professor
    graph=mult
    graph_cut = graph > threshold #(graph_cut fica com 0s e 1s)
    clean_graph = np.multiply(graph, graph_cut) #(a multiplicação vai fazer reset aos edges abaixo do threshold)
    clean_graph = np.multiply(graph, (graph > threshold))
    print(mult.todense())
    mult=clean_graph
    #aplicar o pagerank ao grafo
    entity_graph=csr_matrix(mult,dtype=float)
    scores= assistant.computePageRank(entity_graph)

    #Dataframe to debug
    #data=pd.DataFrame(mult.todense(),columns=entitys)
    #data.insert(0,"Entitys",entitys)
    #print(data)

    return entity_graph,transpost_ocurrences,entitys,scores

Imagens

In [ ]:
def saveGraphImages(mult,scores,entitys,topicNumber,answerNumber,indice_modo):
  modos=["1-Normal","2-Entidades_da_primeira","3-Entidades_todas_perguntas"
          ,"4-Função_penalidade"]
  topico="Topico_"+str(topicNumber)
  modo=str(modos[indice_modo])
  pergunta="Pergunta_"+str(answerNumber)
  dirName="PW/EntityGraphs/"+modo+"/"+topico+"/"
  #dirName="PW/Isolado/"+topico+"/"
  !mkdir -p $dirName
  image = svg_graph(mult, scores=np.log(scores), names = entitys,filename=dirName+pergunta)
  SVG(image)
  return image

Main

In [ ]:
#Para todas as conversas
allx=[]
answersAll = []
allR = []
modo2Reordered=[]
aux = ["31","32","33","34","37","40","49","50","54","56","58","59","61","67","68","69","75","77","78","79"]
#try:
 #   new_file=open("sumariosModo1.txt",mode="w",encoding="utf-8")
#except IOError:
 #   print("File not found or path is incorrect")

for conv in range(0,len(allMatrixAllDataset)): #de 0 a 20 conversas
  
  #if conv not in (0,): # ESCOLHER OS TOPICOS AQUI
      #continue

  modo0 = False # só normal
  modo1 = False #só 1º questão
  #modo2 = automático
  #modo3 = mudar o pickle 
  treshold_1_num_entidades=3

  conversa_topico=allMatrixAllDataset[conv]
  counterRespostas=0
  answersConv=[]
  reorderedConv = []
  for j in  range(0,len(conversa_topico)): #Para cada conteudo->set de (pergunta+respostas) num topico

  #for conteudo in  conversa_topico: #Para cada conteudo->set de (pergunta+respostas) num topico

    #1ºPasso: matrix de entidades e pagerank
    if j == 0:
      entity_graph, transpost_ocurrences, entitys, scores = create_EntityMatrix_Transpose_Pagerank(conversa_topico[j],treshold_1_num_entidades)
    else:
      if modo0:
        entity_graph, transpost_ocurrences, entitys, scores = create_EntityMatrix_Transpose_Pagerank(conversa_topico[j],treshold_1_num_entidades)
      if modo1:
        entity_graph, transpost_ocurrences, entitys, scores = create_EntityMatrix_with1Q_Transpose_Pagerank(conversa_topico[j], [conversa_topico[0]],treshold_1_num_entidades)
      else:
        entity_graph, transpost_ocurrences, entitys, scores = create_EntityMatrix_with1Q_Transpose_Pagerank(conversa_topico[j], conversa_topico[:j],treshold_1_num_entidades)
    print("entity_graph")
    print(len(entity_graph.todense()))
    #1ºPasso (Opcional): criar imagens do grafo de entidades
    
    #image=saveGraphImages(entity_graph,scores,entitys,conv,counterRespostas,1)

    #2ºPasso: calcular top10
    reordered = calcTop10(transpost_ocurrences,scores,conv,counterRespostas)
    allR.append(scores)

    reorderedConv.append(reordered.index)
    #3ºPasso: calcular respostas sumarizada final
    #summary_text = sumarize(reordered)
    #new_file.write(str(aux[conv])+"_"+str(j)+"\n")
    #new_file.write(summary_text+"\n")
    #answersConv.append(summary_text)
    counterRespostas+=1
    
  modo2Reordered.append(reorderedConv)
  #answersAll.append(answersConv)
#new_file.close()

## Calcular pontuacoes

In [ ]:
f = open("/content/PW/Extras/2019qrels.txt", "r")
evaluated = {}
for x in f:
  y = x.split()
  try:
    kk = evaluated[y[0].split("_")[0]]
  except:
    kk = {}
  try:
    vec = kk[y[0].split("_")[1]]
  except:
    vec = {} 
  vec[y[2]]=int(y[3])
  kk[y[0].split("_")[1]] = vec
  evaluated[y[0].split("_")[0]] = kk
  

In [ ]:
import json
with open('/content/PW/Extras/allIn2.json', 'r') as j:
    json_data = json.load(j)

In [ ]:
def soma (allReordered,json_data,evaluated):
  sum = 0
  for k in range(3):
      #pos = allReordered[counter][int(j)-1][k]
      pos = allReordered[k]
      #cod = json_data[i][j]["top10"][pos]
      cod = json_data[pos]
      try:
        #sum = sum +  evaluated[i][j][cod]
        sum = sum +  evaluated[cod]
      except:
        #return -1
        continue
  return sum
    


In [ ]:
def calcMax(arr,soma):
    res = ""
    maxi = max(arr)
    if maxi == 0:
      return "0",soma
    if maxi == arr[0]:
      res = res + "modo0_"
      soma[0] = soma[0]+1
    if maxi == arr[1]:
      res = res + "modo1_"
      soma[1] = soma[1]+1
    if maxi == arr[2]:
      res = res + "modo2_"
      soma[2] = soma[2]+1
    if maxi == arr[3]:
      res = res + "modo3"
      soma[3] = soma[3]+1
    return res, soma

In [ ]:
import csv  

header = ['modo0','modo1', 'modo2', 'modo3']
#header = ['pergunta', 'modo0','modo1', 'modo2', 'modo3']
somas = [0] * 4
with open('resultados3.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    counter = 0
    for i in json_data:
      print(i)
      somas = [0] * 4
      for j in json_data[i]:
        print(j)
        try:
          modo0 = soma(modo0Reordered[counter][int(j)-1],json_data[i][j]["top10"],evaluated[i][j])
          modo1 = soma(modo1Reordered[counter][int(j)-1],json_data[i][j]["top10"],evaluated[i][j])
          modo2 = soma(modo2Reordered[counter][int(j)-1],json_data[i][j]["top10"],evaluated[i][j])
          modo3 = soma(modo3Reordered[counter][int(j)-1],json_data[i][j]["top10"],evaluated[i][j])
          comp = [modo0,modo1,modo2,modo3,modo4]
          if -1 not in comp:
            maxIs,somas = calcMax(comp,somas)
            print(comp)
            data = [i+'_'+j,modo0,modo1,modo2,modo3,maxIs]
            #writer.writerow(data)
          #else:
            #print("hi")   
        except:
          print("here")
          continue
      #writer.writerow(somas)
      counter = counter + 1
    # write the data
print(somas)
f.close()
    

## 4º Passo

### Guardar o Sumarization num txt

Guardar num ficheiro txt as respostas

In [ ]:
!mkdir -p PW/SumarioTxt
count=0
with open("PW/SumarioTxt/AllSumarized.txt", "w") as txt_file:
    for topic in answersAll:
        txt_file.write("---------------------" + "\n")
        txt_file.write("Topico " + str(count) + "\n") # works with any number of elements in a line
        for ans in topic:
          txt_file.write("".join(ans) + "\n") # works with any number of elements in a line
    count+=1
    txt_file.write("---------------------" + "\n")